In [2]:
!pip install tensorflow --upgrade
!pip install keras --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.6 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstalled tensorboard-2.14.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.14.0
    Uninstalling tensorflow-2.14.0:
      Successfully

IMPORTING LIBRARIES

In [3]:
#importing modules
import pandas as pd
import os
import sklearn
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree, metrics
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from google.colab import drive
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import joblib

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Telco_Customer_Churn_Dataset.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


USING FEATURE IMPORTANCE FOR FEATURE EXTRACTION

In [5]:
numeric_cols = ['SeniorCitizen', 'tenure', 'MonthlyCharges']
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# Replace NaN values in object columns with 'Unknown'
object_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
               'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
               'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
               'PaperlessBilling', 'PaymentMethod', 'TotalCharges', 'Churn']
df[object_cols] = df[object_cols].fillna('Unknown')

# Ensure 'TotalCharges' column is of numeric type after replacing NaN values
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)

In [6]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


In [7]:
# Separate features and target variable
X = df.drop(['customerID', 'Churn'], axis=1)
y = df['Churn']



In [8]:
from sklearn.compose import ColumnTransformer
# Identify categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Apply one-hot encoding to categorical columns
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(), categorical_cols)],
    remainder='passthrough'
)

X = preprocessor.fit_transform(X)

In [9]:

# Initialize and train the RandomForestClassifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X, y)

# Get feature importances
feature_importances = clf.feature_importances_



In [10]:
feature_importances = clf.feature_importances_

# Create a DataFrame to display feature names and their importance scores
feature_importance_df = pd.DataFrame({'Feature': preprocessor.get_feature_names_out(), 'Importance': feature_importances})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print feature importances
print(feature_importance_df)

                                         Feature  Importance
44                       remainder__TotalCharges    0.164699
42                             remainder__tenure    0.140393
43                     remainder__MonthlyCharges    0.140184
32                  cat__Contract_Month-to-month    0.047904
14                        cat__OnlineSecurity_No    0.032448
39           cat__PaymentMethod_Electronic check    0.029351
23                           cat__TechSupport_No    0.027495
12              cat__InternetService_Fiber optic    0.027489
41                      remainder__SeniorCitizen    0.020156
34                        cat__Contract_Two year    0.019332
0                             cat__gender_Female    0.017813
1                               cat__gender_Male    0.017265
17                          cat__OnlineBackup_No    0.017214
35                      cat__PaperlessBilling_No    0.015235
36                     cat__PaperlessBilling_Yes    0.014799
3                       

In [11]:
selected_columns = ['Churn','SeniorCitizen','gender', 'tenure','InternetService','OnlineSecurity','StreamingTV','StreamingMovies','Contract','PaymentMethod','PaperlessBilling','MonthlyCharges','TotalCharges']

DATA PREPROCESSING

In [12]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Telco_Customer_Churn_Dataset.csv', usecols=selected_columns)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   tenure            7043 non-null   int64  
 3   InternetService   7043 non-null   object 
 4   OnlineSecurity    7043 non-null   object 
 5   StreamingTV       7043 non-null   object 
 6   StreamingMovies   7043 non-null   object 
 7   Contract          7043 non-null   object 
 8   PaperlessBilling  7043 non-null   object 
 9   PaymentMethod     7043 non-null   object 
 10  MonthlyCharges    7043 non-null   float64
 11  TotalCharges      7043 non-null   object 
 12  Churn             7043 non-null   object 
dtypes: float64(1), int64(2), object(10)
memory usage: 715.4+ KB


In [13]:
df.head()

,gender,SeniorCitizen,tenure,InternetService,OnlineSecurity,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,1,DSL,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,34,DSL,Yes,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,2,DSL,Yes,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,45,DSL,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,2,Fiber optic,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


FEATURE AUGMENTATION

In [14]:
#Combine 'StreamingTV' and 'StreamingMovies' into a new feature 'Streaming'
df['Streaming'] = df['StreamingTV'].apply(lambda x: 1 if x == 'Yes' else 0) + df['StreamingMovies'].apply(lambda x: 1 if x == 'Yes' else 0)

# Drop the original 'StreamingTV' and 'StreamingMovies' columns
df.drop(['StreamingTV', 'StreamingMovies'], axis=1, inplace=True)

In [15]:
# Convert 'TotalCharges' column to numeric
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Fill missing values with the mean of the column
mean_total_charges = df['TotalCharges'].mean()
df['TotalCharges'] = df['TotalCharges'].fillna(mean_total_charges)


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   tenure            7043 non-null   int64  
 3   InternetService   7043 non-null   object 
 4   OnlineSecurity    7043 non-null   object 
 5   Contract          7043 non-null   object 
 6   PaperlessBilling  7043 non-null   object 
 7   PaymentMethod     7043 non-null   object 
 8   MonthlyCharges    7043 non-null   float64
 9   TotalCharges      7043 non-null   float64
 10  Churn             7043 non-null   object 
 11  Streaming         7043 non-null   int64  
dtypes: float64(2), int64(3), object(7)
memory usage: 660.4+ KB


In [17]:
df.head()

,gender,SeniorCitizen,tenure,InternetService,OnlineSecurity,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Streaming
0,Female,0,1,DSL,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,0
1,Male,0,34,DSL,Yes,One year,No,Mailed check,56.95,1889.50,No,0
2,Male,0,2,DSL,Yes,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,0
3,Male,0,45,DSL,Yes,One year,No,Bank transfer (automatic),42.30,1840.75,No,0
4,Female,0,2,Fiber optic,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,0


In [18]:
#Encoding Churn column
df = pd.get_dummies(df, columns=['Churn'], drop_first=True)
df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   tenure            7043 non-null   int64  
 3   InternetService   7043 non-null   object 
 4   OnlineSecurity    7043 non-null   object 
 5   Contract          7043 non-null   object 
 6   PaperlessBilling  7043 non-null   object 
 7   PaymentMethod     7043 non-null   object 
 8   MonthlyCharges    7043 non-null   float64
 9   TotalCharges      7043 non-null   float64
 10  Streaming         7043 non-null   int64  
 11  Churn_Yes         7043 non-null   uint8  
dtypes: float64(2), int64(3), object(6), uint8(1)
memory usage: 612.3+ KB


SCALING NUMERICAL DATA

In [19]:
from sklearn.preprocessing import StandardScaler

# Select the numerical features
numerical_features = ['SeniorCitizen', 'tenure','Streaming', 'MonthlyCharges', 'TotalCharges']

# Initialize the StandardScaler
scaler = StandardScaler()

# Scale the numerical features
df[numerical_features] = scaler.fit_transform(df[numerical_features])

ONE-HOT ENCODING CATEGORICAL DATA

In [20]:
from sklearn.preprocessing import OneHotEncoder

# Select categorical columns for one-hot encoding
categorical_cols = ['gender','PaymentMethod','InternetService', 'OnlineSecurity', 'Contract', 'PaperlessBilling']

# Initialize the OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse=False)

# Apply one-hot encoding to the selected categorical columns
encoded_features = encoder.fit_transform(df[categorical_cols])

# Create a DataFrame with the encoded features
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))

# Concatenate the encoded features with the original DataFrame
df = pd.concat([df, encoded_df], axis=1)

# Drop the original categorical columns
df.drop(categorical_cols, axis=1, inplace=True)

df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 17 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   SeniorCitizen                          7043 non-null   float64
 1   tenure                                 7043 non-null   float64
 2   MonthlyCharges                         7043 non-null   float64
 3   TotalCharges                           7043 non-null   float64
 4   Streaming                              7043 non-null   float64
 5   Churn_Yes                              7043 non-null   uint8  
 6   gender_Male                            7043 non-null   float64
 7   PaymentMethod_Credit card (automatic)  7043 non-null   float64
 8   PaymentMethod_Electronic check         7043 non-null   float64
 9   PaymentMethod_Mailed check             7043 non-null   float64
 10  InternetService_Fiber optic            7043 non-null   float64
 11  Inte

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Streaming,Churn_Yes,gender_Male,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes
0,-0.439916,-1.277445,-1.160323,-0.994971,-0.905859,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.439916,0.066327,-0.259629,-0.173876,-0.905859,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,-0.439916,-1.236724,-0.362660,-0.960399,-0.905859,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,-0.439916,0.514251,-0.746535,-0.195400,-0.905859,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,-0.439916,-1.236724,0.197365,-0.941193,-0.905859,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [21]:
# Separate features and target variable
X = df.drop(['Churn_Yes'], axis=1)
y = df['Churn_Yes']

X


,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Streaming,gender_Male,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes
0,-0.439916,-1.277445,-1.160323,-0.994971,-0.905859,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.439916,0.066327,-0.259629,-0.173876,-0.905859,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,-0.439916,-1.236724,-0.362660,-0.960399,-0.905859,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,-0.439916,0.514251,-0.746535,-0.195400,-0.905859,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,-0.439916,-1.236724,0.197365,-0.941193,-0.905859,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,-0.439916,-0.340876,0.665992,-0.129281,1.440147,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
7039,-0.439916,1.613701,1.277533,2.242808,1.440147,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
7040,-0.439916,-0.870241,-1.168632,-0.855182,-0.905859,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
7041,2.273159,-1.155283,0.320338,-0.872777,-0.905859,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


TRAINING MLP MODEL

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Initialize individual MLP models
mlp = MLPClassifier(max_iter=1000, random_state=42)

# Define parameter grid for grid search
param_grid = {
    'hidden_layer_sizes': [(100, 50), (50, 25), (75, 30)],
    'alpha': [0.0001, 0.001, 0.01],
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)

# Perform grid search on the training data
grid_search.fit(X_train, y_train)

# Get the best model from the grid search
best_mlp = grid_search.best_estimator_

# Print the best parameters found by grid search
print("Best Parameters:", grid_search.best_params_)

# Make predictions using the best model
y_pred_best = best_mlp.predict(X_test)

# Calculate accuracy
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f'Best Model Accuracy: {accuracy_best:.2f}')

In [ ]:
X.info()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [ ]:
# Initialize individual MLP models
mlp1 = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
mlp2 = MLPClassifier(hidden_layer_sizes=(50, 25), max_iter=1000, random_state=42)
mlp3 = MLPClassifier(hidden_layer_sizes=(75, 30), max_iter=1000, random_state=42)

# Create a VotingClassifier ensemble
ensemble_clf = VotingClassifier(estimators=[
    ('mlp1', mlp1),
    ('mlp2', mlp2),
    ('mlp3', mlp3)
], voting='soft')

# Train the ensemble model
ensemble_clf.fit(X_train, y_train)

# Make predictions using the ensemble model
y_pred_ensemble = ensemble_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_ensemble)
print(f'Accuracy: {accuracy:.2f}')

# Calculate AUC score
auc_score = roc_auc_score(y_test, y_pred_ensemble)
print(f'AUC Score: {auc_score:.2f}')

# Evaluate the ensemble model
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
print(f'Ensemble Model Accuracy: {accuracy_ensemble:.2f}')

TRAINING MLP MODEL USING THE KERAS FUNCTIONAL API

In [ ]:


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the input layer
input_layer = Input(shape=(X_train_scaled.shape[1],))

# Define hidden layers
hidden1 = Dense(units=300, activation='tanh')(input_layer)
hidden2 = Dense(units=200, activation='tanh')(hidden1)
hidden3 = Dense(units=150, activation='tanh')(hidden2)
hidden4 = Dense(units=100, activation='tanh')(hidden3)
hidden5 = Dense(units=75, activation='tanh')(hidden4)
hidden6 = Dense(units=50, activation='tanh')(hidden5)
hidden7 = Dense(units=25, activation='tanh')(hidden6)

# Define output layer
output_layer = Dense(units=1, activation='sigmoid')(hidden7)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=100, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Loss: {loss:.2f}')
print(f'Accuracy: {accuracy:.2f}')

# Predict probabilities for the positive class
y_pred_proba = model.predict(X_test_scaled)

# Calculate AUC score
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f'AUC Score: {auc_score:.2f}')


Hyperparameter Tuning for Multi-Layer Perceptron (MLP) Classifier

In [ ]:


# Standardize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the KerasClassifier-compatible function
def create_model(hidden_layers=(300, 200, 150, 100, 75, 50, 25), learning_rate=0.0001):
    model = MLPClassifier(hidden_layer_sizes=hidden_layers, learning_rate_init=learning_rate, max_iter=1000)
    return model

# Define hyperparameters grid
param_grid = {
    'hidden_layer_sizes': [(300, 200, 150, 100, 75, 50, 25), (200, 100, 50), (100,)],
    'learning_rate_init': [0.0001, 0.001, 0.01],
}

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=create_model(), param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)

# Perform grid search on the entire dataset (X_scaled, y)
grid_search.fit(X_scaled, y)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Print the best parameters found by grid search
print("Best Parameters:", grid_search.best_params_)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=42)

# Evaluate the best model on the test set
accuracy_best = best_model.score(X_test, y_test)
print(f'Best Model Accuracy: {accuracy_best:.2f}')


In [ ]:
# Evaluate the best model on the test set
y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Predict probabilities for positive class
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f'AUC Score: {auc_score:.2f}')

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np
import pandas as pd

# Split the data into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.01, random_state=42)  # 0.25 x 0.8 = 0.2

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define the KerasClassifier-compatible function
def create_model(hidden_layers=(150, 100, 75, 50, 25),
                activation='relu',
                solver='adam',
                alpha=0.0001,
                learning_rate='constant',
                learning_rate_init=0.0001,
                batch_size=32):
    model = MLPClassifier(hidden_layer_sizes=hidden_layers,
                          activation=activation,
                          solver=solver,
                          alpha=alpha,
                          learning_rate=learning_rate,
                          learning_rate_init=learning_rate_init,
                          batch_size=batch_size,
                          max_iter=1000)
    return model

# Define hyperparameters grid
param_grid = {
    'hidden_layer_sizes': [(100, 75, 50, 25), (200, 100, 50), (100,)],
    'activation': ['tanh'],
    'solver': ['adam', 'sgd', 'lbfgs'],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'learning_rate_init': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive'],
    'batch_size': [32, 64, 128]
}

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=create_model(), param_grid=param_grid, cv=3, scoring='roc_auc', n_jobs=-1)

# Perform grid search on the entire dataset (X_train_scaled, y_train)
grid_search.fit(X_train_scaled, y_train)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Print the best parameters found by grid search
print("Best Parameters:", grid_search.best_params_)

# Evaluate the best model on the test set
y_pred_proba = best_model.predict_proba(X_test_scaled)[:, 1]  # Predict probabilities for positive class
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f'AUC Score on Test Set: {auc_score:.2f}')

# Implement Cross-validation to calculate training and validation loss, accuracy
cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=5, scoring='roc_auc')
print(f'Cross-validation AUC scores: {cv_scores}')
print(f'Mean AUC from cross-validation: {np.mean(cv_scores):.2f} (+/- {np.std(cv_scores):.2f})')

# Train the best model on the entire training set to get training/validation metrics
best_model.fit(X_train_scaled, y_train)
train_loss = best_model.loss_
train_accuracy = accuracy_score(y_train, best_model.predict(X_train_scaled))
val_loss = best_model.score(X_val_scaled, y_val)
val_accuracy = accuracy_score(y_val, best_model.predict(X_val_scaled))

print(f"Training Loss: {train_loss:.2f}, Validation Loss: {val_loss:.2f}")
print(f"Training Accuracy: {train_accuracy:.2f}, Validation Accuracy: {val_accuracy:.2f}")


Training and Validation Loss Comparison Over Epochs

In [ ]:
import matplotlib.pyplot as plt
history = best_model.fit(X_train_scaled, y_train)

train_loss = history.loss_curve_
val_loss = best_model.score(X_val_scaled, y_val)

# Plot training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(train_loss, label='Training Loss')
plt.axhline(y=val_loss, color='r', linestyle='--', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import joblib
import pickle

# Save the OneHotEncoder object
with open('/content/drive/My Drive/Colab Notebooks/one_hot_encoder.pkl', 'wb') as file:
    joblib.dump(encoder, file)

# saving the scaler object using joblib for later use in ensemble models
joblib.dump(scaler, '/content/drive/My Drive/Colab Notebooks/scaler.pkl')

# Save the model
model.save('/content/drive/My Drive/Colab Notebooks/best_model.h5')